<a href="https://colab.research.google.com/github/jaydeepthik/sequence-modeling-with-bidirectional-RNNs-LSTMs-1D-convolutions-etc/blob/master/CNN_sentiment_analysis_tfl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tflearn
from  tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_avg_pool, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import pad_sequences
from tflearn.datasets import imdb
import numpy as np

In [0]:
train, test, _ = imdb.load_data(n_words=10000)
X_train, y_train = train
X_test, y_test = test

y_train = np.reshape(y_train, [-1,1])
y_test = np.reshape(y_test, [-1,1])

X_train = pad_sequences(X_train, maxlen=100)
X_test = pad_sequences(X_test, maxlen=100)

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((22500, 100), (2500, 100), (22500, 1), (2500, 1))

In [21]:
tf.reset_default_graph()
net = input_data(shape=[None, 100], name='input_data')
net = tflearn.layers.embedding(net, input_dim=10000, output_dim=300)

#conv branch1
branch1 = conv_1d(net, 100, 3, 1, padding='valid', activation='relu', regularizer='L2')
#conv branch2
branch2 = conv_1d(net, 100, 4, 1, padding='valid', activation='relu', regularizer='L2')
#conv branch3
branch3 = conv_1d(net, 100, 5, 1, padding='valid', activation='relu', regularizer='L2')

net = merge([branch1, branch2, branch3], axis=1, mode='concat')
net = tf.expand_dims(net, 2)
net = global_max_pool(net)
net = dropout(net, keep_prob=0.50)
net = fully_connected(net, 1, activation='sigmoid')

net = regression(net, optimizer='adam', loss='binary_crossentropy')

model  = tflearn.DNN(net)
model.fit(X_train, y_train, n_epoch=10, shuffle=True, validation_set=(X_test, y_test), show_metric=True, batch_size=50)

Training Step: 4499  | total loss: 0.55445 | time: 23.164s
| Adam | epoch: 010 | loss: 0.55445 - binary_acc: 0.6934 -- iter: 22450/22500
Training Step: 4500  | total loss: 0.55461 | time: 24.225s
| Adam | epoch: 010 | loss: 0.55461 - binary_acc: 0.7020 | val_loss: 0.63038 - val_acc: 0.6552 -- iter: 22500/22500
--
